In [1]:
#!/usr/bin/python3
import argparse
from random import uniform, randint

def generate_problem(
    variables=10, 
    constraint_probability=0.25, 
    R=100,
    min_intervals=1, 
    max_intervals=5,
    scaling_factor=1,
    ):
    
    T_all = []
    
    for j in range(variables):
        T_all.append({
            'i': 0,
            'j': j,
            'intervals': [(-R, R)],
        })
        
    for i in range(1, variables):
        for j in range(i+1, variables):
            if uniform(0, 1) < constraint_probability:
                num_intervals = randint(min_intervals, max_intervals)
                points = [randint(-R, R) for _ in range(2 * num_intervals)]
                points.sort()
                
                intervals_temp = list(zip(points[::2], points[1::2]))
                
                # apply scaling factor:
                intervals = []
                for a, b in intervals_temp:
                    midpoint = int((a+b)/2)
                    l = int( midpoint - (midpoint-a)*scaling_factor )
                    r = int( midpoint + (b-midpoint)*scaling_factor )
                    intervals.append( (l, r) )
                    
                T_all.append({
                    'i': i,
                    'j': j,
                    'intervals': intervals,
                })

    return T_all

In [78]:
def verify_witness(X, T):
    failed_constraints = []
    for constraint in T:
        i = constraint['i']
        j = constraint['j']

        diff = X[j] - X[i]

        passed = False
        for interval in constraint['intervals']:
            if interval[0] <= diff <= interval[1]:
                passed = True
                break
        
        if not passed:
            failed_constraints += [f"X_i={X[i]}, X_j={X[j]} on {constraint}"]

    return failed_constraints


def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

   
def purely_random(T, r, iterations):
    best_gene = None
    best_gene_failed = None
    
    for i in range(iterations):
        gene = random_gene(T, r)
        gene_failed = verify_witness(gene, T)
        if not best_gene or len(gene_failed) < len(best_gene_failed):
            best_gene = gene
            best_gene_failed = gene_failed
        
        if len(gene_failed) == 0:
            break
            
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [89]:
T = generate_problem(15, scaling_factor=1)
purely_random(T, 100, 1000)

num iterations: 1000
best gene: [0, -59, -32, 10, 17, 36, -45, -34, 75, 14, -85, 26, -59, -56, -23]
constraints failed: 7 out of: 37
failed constraints: ["X_i=-59, X_j=10 on {'i': 1, 'j': 3, 'intervals': [(-32, 23)]}", "X_i=-32, X_j=36 on {'i': 2, 'j': 5, 'intervals': [(-100, -90), (-60, -21), (-4, 62), (78, 96)]}", "X_i=-32, X_j=75 on {'i': 2, 'j': 8, 'intervals': [(-77, -56)]}", "X_i=17, X_j=-34 on {'i': 4, 'j': 7, 'intervals': [(-96, -69), (-29, 14), (26, 27), (65, 70), (73, 85)]}", "X_i=-34, X_j=-23 on {'i': 7, 'j': 14, 'intervals': [(-99, -79), (-63, -33), (-28, 6), (37, 58), (59, 83)]}", "X_i=-85, X_j=26 on {'i': 10, 'j': 11, 'intervals': [(-80, -64), (-34, -9), (38, 39), (71, 79)]}", "X_i=26, X_j=-56 on {'i': 11, 'j': 13, 'intervals': [(-44, -33), (-14, 4), (9, 27)]}"]
